In [15]:
import cv2

if hasattr(cv2, "aruco"):
    print("Aruco is available.")


Aruco is available.


In [16]:
import cv2
import numpy as np

cap = cv2.VideoCapture("aruco.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("aruco_green_overlay.mp4", fourcc, fps, (frame_w, frame_h))

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

expected_ids = [0, 1, 2, 3]
marker_size = 80
margin = 10


def shift_corner(corner, dx, dy):
    return np.array([corner[0] + dx, corner[1] + dy])


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    overlay = frame.copy()
    corners, ids, _ = detector.detectMarkers(frame)

    if ids is not None and len(ids) >= 4:
        detected = {int(id_): corners[i][0] for i, id_ in enumerate(ids.flatten()) if int(id_) in expected_ids}

        if all(k in detected for k in expected_ids):
            # Use known corners and apply outward offset
            # Marker 0: use corner 0 (top-left) -> shift (-margin, -margin)
            # Marker 1: use corner 1 (top-right) -> shift (+margin, -margin)
            # Marker 2: use corner 2 (bottom-right) -> shift (+margin, +margin)
            # Marker 3: use corner 3 (bottom-left) -> shift (-margin, +margin)

            tl = shift_corner(detected[0][0], -margin, -margin)
            tr = shift_corner(detected[1][1], margin, -margin)
            br = shift_corner(detected[2][2], margin, margin)
            bl = shift_corner(detected[3][3], -margin, margin)

            screen_pts = np.array([tl, tr, br, bl], dtype=np.int32)

            # Draw green overlay
            green_overlay = np.zeros_like(frame)
            cv2.fillPoly(green_overlay, [screen_pts], (0, 255, 0))

            alpha = 0.3
            overlay = cv2.addWeighted(frame, 1 - alpha, green_overlay, alpha, 0)

            # Draw polygon outline
            cv2.polylines(overlay, [screen_pts], True, (0, 255, 0), 2)

    out.write(overlay)

cap.release()
out.release()
print("Done. Saved to aruco_green_overlay.mp4")


Done. Saved to aruco_green_overlay.mp4
